In [ ]:
# 라벨링

from sklearn.preprocessing import LabelEncoder
 
# encode class values as integers
encoder = LabelEncoder()

encoder.fit(data['FLO'])
data['FLO'] = encoder.transform(data['FLO'])
encoder.fit(data['FLT'])
data['FLT'] = encoder.transform(data['FLT'])
encoder.fit(data['SDT_DY'])
data['SDT_DY'] = encoder.transform(data['SDT_DY'])
encoder.fit(data['AOD'])
data['AOD'] = encoder.transform(data['AOD'])
encoder.fit(data['IRR'])
data['IRR'] = encoder.transform(data['IRR'])
encoder.fit(data['DLY'])
data['DLY'] = encoder.transform(data['DLY'])

# ----------------------------------------------------------

# 학습

y= data['DLY']
X =data.drop('DLY', axis= 1)
X

from imblearn.over_sampling import ADASYN 
sm = ADASYN()
X, y = sm.fit_resample(X, y)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.loc[:, data.columns != 'DLY'], data['DLY'], random_state=0)

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))


#-------------------------------------------

# PDP

# Partial Dependence Plots - adapted from Dan B NB on Kaggle()
import pandas as pd
from pandas import read_csv, DataFrame
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np


from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

# get_some_data is defined in hidden cell above.
# scikit-learn originally implemented partial dependence plots only for Gradient Boosting models
# this was due to an implementation detail, and a future release will support all model types.
my_model = GradientBoostingRegressor()
# fit the model as usual
temp_x = X[:]
temp_x.drop('STTATT', axis=1)
my_model.fit(temp_x, y)
# Here we make the plot

temp_x

temp_x.columns.values.tolist()

plt.rcParams["figure.figsize"] = (20,30)
my_plots = plot_partial_dependence(my_model,
                                   features=[i for i in range(12,len(temp_x.columns.values.tolist()))],
                                   feature_names=temp_x.columns.values.tolist(), # column numbers of plots we want to show
                                   X=temp_x,            # raw predictors data.
                                   grid_resolution=100000) # number of values to plot on x axis